# Question 2

## a)

State is represented as follows:

Idx 0: Missionaries on left

Idx 1: Cannibals on left

Idx 2: Boats on left

Idx 3: Missionaries on right

Idx 4: Cannibals on right

Idx 5: Boats on right

In [10]:
import time

# Goal state:
startState = [3, 3, 1, 0, 0, 0]
goalState = [0, 0, 0, 3, 3, 1]
legalMoves = [[-1, 0, -1, 1, 0, 1],
              [0, -1, -1, 0, 1, 1],
              [-1, -1, -1, 1, 1, 1],
              [-2, 0, -1, 2, 0, 1],
              [0, -2, -1, 0, 2, 1],
              [1, 0, 1, -1, 0, -1],
              [0, 1, 1, 0, -1, -1],
              [1, 1, 1, -1, -1, -1],
              [2, 0, 1, -2, 0, -1],
              [0, 2, 1, 0, -2, -1]]

def takeMove(state, move):
    newState = state.copy()
    for idx in range(len(state)):
        newState[idx] = state[idx] + move[idx]
    return newState

def isValidState(state):
    # Check for negative counts of ppl / boats
    for x in state:
        if x < 0:
            return False
    # Check for cannibals outnumbering missionaries.
    # If there are no missionarries they are not outnumbered.
    if (state[0] < state[1] and state[0] != 0) or (state[3] < state[4] and state[3] != 0):
        return False
    # Redundant, but checking for more than one boat
    if state[2] > 1 or state[5] > 1:
        return False
    return True
    
# This problem can't be reliably solved with DFS because of infinite loops

# Really weird implementation of BFS
# Each level of the game tree is stored in a 2d list for backtracking
# Returns the states visited to reach the goal.
def bfs(state):
    # Will store tuples of state and parent index
    # works as a reverse only tree for backtracking
    statesInLevel = []
    statesInLevel.append([(state, -1)]) # Level 0
    level = 1
    foundGoal = False
    while(not foundGoal):
        parentIdx = 0
        # Creating the new level
        statesInLevel.append([])
        # Loop over each state in previous level and add branches to next level
        for s in statesInLevel[level - 1]:
            # Branch for every legal move
            for move in legalMoves:
                newState = takeMove(s[0], move)
                if isValidState(newState):
                    statesInLevel[level].append((newState, parentIdx))
                    if newState == goalState:
                        foundGoal = True
                        break
                
            if foundGoal:
                break
            parentIdx += 1       
        level += 1
    # Found the goal state: time to backtrack
    ans = []
    level = len(statesInLevel) - 1
    idx = len(statesInLevel[level]) - 1
    while level >= 0:
        ans.insert(0, statesInLevel[level][idx][0])
        idx = statesInLevel[level][idx][1]
        level -= 1
        
    return ans
startTime = time.time()
ans = bfs(startState)
endTime = time.time()
for s in ans:
    print(s)

print(endTime - startTime, "seconds")

[3, 3, 1, 0, 0, 0]
[2, 2, 0, 1, 1, 1]
[3, 2, 1, 0, 1, 0]
[3, 0, 0, 0, 3, 1]
[3, 1, 1, 0, 2, 0]
[1, 1, 0, 2, 2, 1]
[2, 2, 1, 1, 1, 0]
[0, 2, 0, 3, 1, 1]
[0, 3, 1, 3, 0, 0]
[0, 1, 0, 3, 2, 1]
[1, 1, 1, 2, 2, 0]
[0, 0, 0, 3, 3, 1]
0.0797877311706543 seconds


Since this problem has equal weights for each move, BFS will give us the an optimal solution

## b)

In [14]:
from queue import PriorityQueue

visitedStates = []

def findState(state):
    idx = 0
    for s in visitedStates:
        if s[0] == state:
            return idx
        idx += 1
    return -1
# Simple heuristic: Returns the number of pieces left on the left side - 1 times 2 - 1
# Each boat cycle can carry two people, but at least one person needs to come back.
# Therefore, every two moves can only bring one person except for the last move
# which can carry two people without a return.
# With n = number of ppl on the left
# n = 6 -> n = 5 with 2 moves
# n = 5 -> n = 4 with 2 moves
# n = 4 -> n = 3 with 2 moves
# n = 3 -> n = 2 with 2 moves
# n = 2 -> n = 0 with 1 move
# To actually solve the problem, you need to shuffle around the missionaries
# and canibals leading to more steps so this will never overestimate.
def heuristic(state):
    h = 2*(state[0] + state[1] - 1) - 1;
    # Accounting for case where boat is on right side.
    if state[5] == 1:
        h += 3
    return h

# Really wierd A* implementation
def AStar(start, goal):
    # Example element of pq: 
    # (distTraveled + estDistRemaining, stateIndex)
    pq = PriorityQueue()
    pq.put((0, 0))
    # Example element of visitedStates:
    # (state, parentStateIndex, cost)
    visitedStates.append((start, -1, 0))
    foundGoal = False
    parentOfGoalIdx = -1
    while(not foundGoal):
        _, parentIdx = pq.get()
        parentCost = visitedStates[parentIdx][2]
        for move in legalMoves:
            newState = takeMove(visitedStates[parentIdx][0], move)
            if isValidState(newState):
                if newState == goal:
                    foundGoal = True
                    parentOfGoalIdx = parentIdx
                    break
                i = findState(newState)
                
                if i == -1: # State has not been visited
                    visitedStates.append((newState, parentIdx, parentCost + 1))
                    pq.put((parentCost + 1 + heuristic(newState), len(visitedStates) - 1))
                else: # State has been visited
                    if parentCost + 1 < visitedStates[i][2]:
                        visitedStates[i] = (newState, parentIdx, parentCost + 1)
                        pq.put((parentCost + 1 + heuristic(newState), i))
    # Found goal state: time to backtrack
    ans = []
    ans.append(goal)
    idx = parentOfGoalIdx
    while idx != -1:
        ans.insert(0, visitedStates[idx][0])
        idx = visitedStates[idx][1]
    return ans
startTime = time.time()
ans = AStar(startState, goalState)
endTime = time.time()
for s in ans:
    print(s)
print(endTime - startTime, "seconds")

[3, 3, 1, 0, 0, 0]
[2, 2, 0, 1, 1, 1]
[3, 2, 1, 0, 1, 0]
[3, 0, 0, 0, 3, 1]
[3, 1, 1, 0, 2, 0]
[1, 1, 0, 2, 2, 1]
[2, 2, 1, 1, 1, 0]
[0, 2, 0, 3, 1, 1]
[0, 3, 1, 3, 0, 0]
[0, 1, 0, 3, 2, 1]
[1, 1, 1, 2, 2, 0]
[0, 0, 0, 3, 3, 1]
0.000997304916381836 seconds
